In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import json
import numpy as np

Open the train data

In [3]:
f = open("drive/MyDrive/data/defi-text-mine-egc-2026/train_v2.jsonl", "r")

text = []
acronym = []
options = []

for line in f:
  try:
      data = json.loads(line)

      text.append(data["text"])

      acronym.append(data["acronym"])

      options.append(data["options"])

  except ValueError:
      print('Invalid input:',line)

f.close()

# Exploratory analysis

Let's visualize few examples

In [4]:
text[:5]

['LRA  limite de résistance des attelages PAR poste d’aiguillage et de régulation PL pleine ligne PN passage à niveau RFN réseau ferré national ',
 '  Désigna -tion des PN ',
 "prédéterminées de trains : _x0001_ les masses admissibles standard pour les trains (et éventuellement celles pour les trains entiers) pour chaque série ou groupement de séries d'EM autorisés à circuler sur tout ou partie du domaine couvert par le document, _x0001_ la limite de résistance des attelages standard [en distinguant éventuellement avec ",
 '/Commentaires N° AC B81500 thermique:  compatibilité : OUI/restrictions : les AGC sont interdits à la desserte commerciale des établis-sements dont la hauteur des quais est ≥ 760 mm ',
 "kilomètres/heure (ex : 12 pour 120 km/h), _x0001_ d'une lettre indiquant le respect ou non des tableaux indicateurs de vitesse de type C (C respecte les TIV, N ne respecte pas les TIV). Engin moteur Véhicule ayant la capacité de se déplacer par ses propres moyens "]

In [5]:
acronym[:5]

['PAR', 'PN', 'EM', 'AC', 'TIV']

In [6]:
options[:5]

[{"Plan d'action régularité": False,
  "Poste d'aiguillage et de régulation : assure les commandes des installations de signalisation et la gestion de la circulation des huit lignes à grande vitesse": True,
  "PONT DE L'ARCHE": False,
  "Plan d'action régional": False},
 {'Passages à niveau : fichier des pn, recensement des accidents de pn, statistiques (infrastructure)': False,
  'Paris Nord': False,
  'PONS': False,
  'Période normale': False,
  'Partiellement noté': False,
  'Pousse non attelée': False,
  'Passage à Niveau': True},
 {'EMERAINVILLE PONTAULT COMBAULT': False,
  'Engin Moteur.   Terme général désignant tout véhicule ayant la propriété de se déplacer par ses propres moyens : machine, automoteur, élément automoteur, draisine, engin spécial motorisé non déraillable.': True,
  'Emetteur': False,
  'État membre': False,
  'Étude mécanique': False,
  'électro-magnétique': False},
 {'ACcès': False,
  "Agent d'aCcompagnement ": False,
  'Agent Circulation  ': False,
  'Agent d

What is the min, max and average number of options?

In [8]:
num_options = []
num_answers = []
for item in options:
    num_answers.append(sum(item.values()))
    num_options.append(len(item.values()))

print(np.min(num_options), np.max(num_options), np.median(num_options))

2 13 4.0


Should be 2, 13 and 4 cosequently

Are there any examples with more than 1 correct option?

In [9]:
print(max(num_answers), sum([i > 1 for i in num_answers]))

2 9


There are max 2 correct answers at the same time. It's only 9 times that there is more than one correct answer -- we could ignore it if needed

In [ ]:
print(sum([i == 0 for i in num_answers]))

68


Hovewer, there are 68 times that no correct answer is given. We should not ignore it in our method if possible

# Semantic similarity method

Baseline method: compute semantic similarity b/w each segment and each option, then select an option with the highest similarity. The method if based on the assumption that there is always 1 correct answer at each segment.

In [ ]:
!pip install sentence_transformers

from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('dangvantuan/french-document-embedding',trust_remote_code=True)

configuration.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/dangvantuan/bilingual_impl:
- configuration.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modelling.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/dangvantuan/bilingual_impl:
- modelling.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Perform prediction

In [ ]:
y_pred = []
y_true = []

for i in range(len(text)):
  text_embedding = model.encode(text[i])

  options_embedding = model.encode(list(options[i].keys()))

  scores = util.dot_score(text_embedding, options_embedding)

  labels_pred = np.zeros(len(options[i]),bool)

  index_max = int(np.argmax(scores.numpy()))
  labels_pred[index_max] = True

  y_pred.append(labels_pred)
  y_true.append(list(options[i].values()))

Evaluation

In [ ]:
from sklearn.metrics import f1_score

f1_list = []

for i in range(len(y_true)):
  f1_list.append(f1_score(y_true[i],y_pred[i],average=None)[1]) # take f1-score of positive class

print(np.mean(f1_list))

0.5799457994579945


Should be 0.5799457994579945, which is not too bad for beginning.